In [9]:
%pip install --upgrade --quiet pymilvus[model] langchain-milvus langchain-openai
%pip install -qU langchain_community pypdf

zsh:1: no matches found: pymilvus[model]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_ollama import OllamaLLM

In [11]:
CONNECTION_URI = "http://localhost:19530"

In [12]:
ollama_embedding_func = OllamaLLM(model="moondream")

In [ ]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Setup
def setup_ollama_pdf_vectorstore(file_path, embedding_model="moondream"):
    try:
        # 1. Load PDF
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()

        # 2. Text Splitting (Optional but recommended)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        splits = text_splitter.split_documents(docs)

        # 3. Ollama Embeddings
        embeddings = OllamaEmbeddings(
            model=embedding_model,  # You can change this to other Ollama models
            num_gpu=16  # Optional: specify GPU usage
        )

        # 4. Create Chroma Vector Store
        vector_store = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            collection_name="ollama_pdf_collection"
        )

        return vector_store

    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None

files = os.listdir("papers")
print(files)

vector_store = setup_ollama_pdf_vectorstore(f"papers/{files[0]}")
print(f"Vector store for {files[0]} created successfully")
print("")

['2303.05510v1.pdf']


In [6]:
# Perform Similarity Search
contexts = ""
query = input("Enter your query: ")

if vector_store:
    results = vector_store.similarity_search(query, k=10)
    
    for doc in results:
        print("\n--- Relevant Document Excerpt ---")
        print(doc.page_content)
        contexts = contexts + doc.page_content


--- Relevant Document Excerpt ---
Published as a conference paper at ICLR 2023
APPENDIX
In this appendix, we supplement the main paper by providing more thorough empirical evaluations to
back up our claims and more detailed descriptions of the algorithms to help readers better understand
our paper.
This appendix is organized as follows.
• In Sec. A, we provide more comprehensive results of our algorithm and the baseline algorithms.
We also include the license information of the datasets we use.
• In Sec. B, we consider the scenario where test cases are not provided. We evaluate our PG-TD
algorithm using automatically-generated test cases.
• In Sec. C, we provide empirical evidence for our claims in the discussion section that our
algorithm is versatile and can be used to optimize different code generation objectives other
than the pass rate. We consider the objectives of generating concise codes and generating
codes with more comments.

--- Relevant Document Excerpt ---
the Transforme

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize Ollama Chat Model
chat_model = ChatOllama(
    model="llama3.2:1b",  # You can change to other models like mistral, phi3
    temperature=0.7,  # Creativity level
    num_gpu=16  # Optional GPU configuration
)

# Simple Chat Interaction
messages = [
    SystemMessage(content="You are a helpful assistant for my research."),
    HumanMessage(content= f"Query + {query} with the context + {contexts}")
]

# Generate Response
response = chat_model.invoke(messages)
print("--- Ollama Response ---")
print(response.content)

--- Ollama Response ---
The text appears to be a research paper or an academic article discussing the results of various machine learning models for code generation, specifically Transformer-based models.

Here's a summary of the key findings:

**Introduction**

The authors introduce their work on using large language models like Transformer for code generation. They discuss how previous studies have achieved significant improvements in benchmarking tasks such as code translation, completion, and problem-solving.

**Background and Related Work**

The authors review existing research on large language models, including those by Vaswani et al. (2017a), Ahmad et al. (2021), Chen et al. (2021a), Hendrycks et al. (2021), Li et al. (2022), Raffel et al. (2020), Roziere et al. (2020), and Roziere et al. (2022).

**Methodology**

The authors describe their approach, which involves:

1. Pre-training a large Transformer model on a large programming corpus.
2. Fine-tuning the pre-trained model fo